In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from helper import gen_I_Q_Amp_mat, gen_I_Q_Amp_Angle_mat

# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using:", device)

using: cuda


In [2]:
from prepare_train_dataset import prepare_train_dataset

train_loader, val_loader = prepare_train_dataset(dim=1)

using: cuda
validate with ['CIR_data_LOS_2_1.npz', 'CIR_data_NLOS_3_2.npz']


In [10]:
# Define the PyTorch model
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        # 定义MLP的层
        self.w = 32
        self.emb = nn.Sequential(
            nn.Linear(1*64, self.w),
            nn.ReLU(),
            nn.Linear(self.w, self.w//4),
            nn.ReLU(),
            nn.Linear(self.w//4, 1),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        # 定义前向传播
        x = self.emb(x)
        x = self.sigmoid(x)
        return x

In [11]:
from helper import count_parameters_bytes

# Create an instance of the PyTorch model and move it to GPU
model = PyTorchModel().to(device)

# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy loss
optimizer = optim.Adam(model.parameters(), lr=1e-6)

count_parameters_bytes(model)

Number of parameters: 9.41 KB


2.353

In [8]:
# Training loop
epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch [{epoch+1}/{epochs}] Train Loss: {total_loss/len(train_loader)}")

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        for batch_x_val, batch_y_val in val_loader:
            val_outputs = model(batch_x_val)
            val_loss += criterion(val_outputs, batch_y_val.view(-1, 1)).item()
            val_predictions = (val_outputs > 0.5).float()
            correct_predictions += (val_predictions == batch_y_val.view(-1, 1)).sum().item()
            total_samples += batch_y_val.size(0)
        val_accuracy = correct_predictions / total_samples
        print(f"Epoch [{epoch+1}/{epochs}] Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {val_accuracy * 100:.2f}%")

# 保存模型
# torch.save(model, 'cnn.pt')

Epoch [1/100] Train Loss: 0.6916107111708621
Epoch [1/100] Validation Loss: 0.692548575678042, Validation Accuracy: 49.21%
Epoch [2/100] Train Loss: 0.6872766969913062
Epoch [2/100] Validation Loss: 0.689064023750169, Validation Accuracy: 49.21%
Epoch [3/100] Train Loss: 0.6793725261643564
Epoch [3/100] Validation Loss: 0.6838397341115134, Validation Accuracy: 55.89%
Epoch [4/100] Train Loss: 0.6665867685015363
Epoch [4/100] Validation Loss: 0.6764572614005634, Validation Accuracy: 60.30%
Epoch [5/100] Train Loss: 0.6475684964545159
Epoch [5/100] Validation Loss: 0.6675367089254516, Validation Accuracy: 63.09%
Epoch [6/100] Train Loss: 0.6219477197046303
Epoch [6/100] Validation Loss: 0.6566887444683484, Validation Accuracy: 65.24%


KeyboardInterrupt: 